# Анализ продаж автомобилей в Казахстане в 2019 году

## Загрузка данных и просмотр общей информации

In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('autokz2019.csv',decimal =',', sep=';', skipinitialspace=True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39966 entries, 0 to 39965
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Год                             39966 non-null  int64  
 1   Месяц                           39966 non-null  object 
 2   Компания                        39966 non-null  object 
 3   Бренд                           39966 non-null  object 
 4   Модель                          39966 non-null  object 
 5   Модификация                     36375 non-null  object 
 6   Год выпуска                     39465 non-null  object 
 7   Страна-производитель            39966 non-null  object 
 8   Вид топлива                     36826 non-null  object 
 9   Объём двиг, л,                  35708 non-null  object 
 10  Коробка передач                 36711 non-null  object 
 11  Тип привода                     35677 non-null  object 
 12  Сегмент                         

удаляем столбцы: наименование дилерского центра, форма расчета, сегмент, сегментация eng, локализация производства, тип клиента и модификация (они либо бесполезны, либо много пропусков)

In [4]:
df = df.drop(df.columns[[5, 12, 14, 15, 16, 23, 24]], axis=1)

In [5]:
df.head()

,Год,Месяц,Компания,Бренд,Модель,Год выпуска,Страна-производитель,Вид топлива,"Объём двиг, л,",Коробка передач,Тип привода,Регион,Количество,"Цена, USD","Продажа, USD",Область,Сегментация 2013,Класс 2013
0,2019,Май,Mercur Auto,Audi,A3,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,28115.00,28115.00,г.Алматы,Легковые автомобили,C класс
1,2019,Август,Mercur Auto,Audi,A3,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,32246.99,32246.99,г.Алматы,Легковые автомобили,C класс
2,2019,Апрель,Mercur Auto,Audi,A4,2018,Германия,Бензин,"1,4",S-Tronic,FWD,Алматы,1.0,32000.00,32000.00,г.Алматы,Легковые автомобили,D класс
3,2019,Июль,Mercur Auto,Audi,A4,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,31929.00,31929.00,г.Алматы,Легковые автомобили,D класс
4,2019,Июль,Mercur Auto,Audi,A4,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,31929.00,31929.00,г.Алматы,Легковые автомобили,D класс


Изменим наименования столбцов

In [6]:
df=df.rename(columns={
    'Год': 'year', 'Месяц': 'month', 'Компания': 'company', 'Бренд': 'brand', 'Модель': 'model', 'Год выпуска': 'year_of_release', \
    'Страна-производитель': 'country_of_manufacture', 'Вид топлива': 'fuel', 'Объём двиг, л,': 'engine_volume', 'Коробка передач': 'transmission_box', \
    'Тип привода': 'type_of_drive', 'Регион': 'region', 'Количество': 'quantity', 'Цена, USD': 'price', 'Продажа, USD': 'sale', \
    'Область': 'oblact', 'Сегментация 2013': 'segment', 'Класс 2013': 'class'})

In [7]:
df.head()

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
0,2019,Май,Mercur Auto,Audi,A3,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,28115.00,28115.00,г.Алматы,Легковые автомобили,C класс
1,2019,Август,Mercur Auto,Audi,A3,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,32246.99,32246.99,г.Алматы,Легковые автомобили,C класс
2,2019,Апрель,Mercur Auto,Audi,A4,2018,Германия,Бензин,"1,4",S-Tronic,FWD,Алматы,1.0,32000.00,32000.00,г.Алматы,Легковые автомобили,D класс
3,2019,Июль,Mercur Auto,Audi,A4,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,31929.00,31929.00,г.Алматы,Легковые автомобили,D класс
4,2019,Июль,Mercur Auto,Audi,A4,2018,Германия,Бензин,"1,4",S-tronic,передний,Алматы,1.0,31929.00,31929.00,г.Алматы,Легковые автомобили,D класс


## Предобработка данных

### Проверка на дубликаты

In [8]:
# проверим датафрейм на наличие явных дубликатов
df.duplicated().sum()

23471

Наличие большого количества дубликатов обусловлено тем, что данные представлены по месяцам. Скорее всего, продажи были проведены в разные дни. Дубли оставляем 

### Проверка на пропуски 

In [9]:
report = df.isna().sum().to_frame()
report = report.rename(columns = {0: 'missing_values'})
report['% of total'] = ((report['missing_values'] / df.shape[0])*100).round(2)
report.sort_values(by = 'missing_values', ascending = False)

,missing_values,% of total
type_of_drive,4289,10.73
engine_volume,4258,10.65
transmission_box,3255,8.14
fuel,3140,7.86
year_of_release,501,1.25
quantity,6,0.02
region,0,0.00
segment,0,0.00
oblact,0,0.00
sale,0,0.00


In [212]:
#df[df['year_of_release'].isna()]

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
65,2019,Июнь,Автоцентр-Бавария,BMW,7 серия,NaN,Германия,Бензин,3,АКПП,Полный,Усть-Каменогорск,1.0,93520.28222,93520.28222,Восточно-Казахстанская область,Легковые автомобили,F класс
9599,2019,Май,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,H350,NaN,Республика Казахстан,Дизель,NaN,NaN,2WD,Алматы,1.0,33833.35184,33833.35184,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
9600,2019,Май,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,H350,NaN,Республика Казахстан,Дизель,NaN,NaN,2WD,Алматы,1.0,33833.35184,33833.35184,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
9601,2019,Май,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,H350,NaN,Республика Казахстан,Дизель,NaN,NaN,2WD,Алматы,1.0,33833.35184,33833.35184,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
9602,2019,Май,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,H350,NaN,Республика Казахстан,Дизель,NaN,NaN,2WD,Алматы,1.0,33833.35184,33833.35184,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39542,2019,Сентябрь,Volkswagen Group Rus,Volkswagen,Teramont,NaN,Российская Федерация,NaN,NaN,NaN,NaN,Костанай,1.0,49655.99915,49655.99915,Костанайская область,Внедорожники,Полноразмерные SUV
39554,2019,Июнь,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,EX8,NaN,Республика Казахстан,NaN,NaN,NaN,NaN,Алматы,1.0,34590.76936,34590.76936,г.Алматы,Коммерческие автомобили,Малотоннажные грузовики
39555,2019,Июнь,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,EX8,NaN,Республика Казахстан,NaN,NaN,NaN,NaN,Алматы,1.0,34590.76936,34590.76936,г.Алматы,Коммерческие автомобили,Малотоннажные грузовики
39556,2019,Июнь,Hyundai Com Trans Kazakhstan,Hyundai Truck & Bus,EX8,NaN,Республика Казахстан,NaN,NaN,NaN,NaN,Алматы,1.0,34590.76936,34590.76936,г.Алматы,Коммерческие автомобили,Малотоннажные грузовики


Так как количество пропусков в столбцах 'quantity' и'year_of_release' невелико, удалим их из датасета

In [217]:
#df=df.dropna(subset=['quantity','year_of_release'])

In [10]:
df.iloc[24999]


year                                               2019
month                                            Январь
company                                  Renault Россия
brand                                           Renault
model                                           Sandero
year_of_release                                    2018
country_of_manufacture             Российская Федерация
fuel                                                1,6
engine_volume                                        MT
transmission_box                               Передний
type_of_drive                                  Передний
region                                          Уральск
quantity                                            1.0
price                                       12476.89637
sale                                        12476.89637
oblact                    Западно-Казахстанская область
segment                             Легковые автомобили
class                                           

## Обработка данных в столбцах

### Год выпуска автомобиля

In [214]:
# посмотрим уникальные значения
#df['year_of_release'].unique()

array(['2018', '2017', '2019', nan, '2016', '2014', '2013', '2011',
       '2\xa0018', '2\xa0019'], dtype=object)

In [85]:
# скомпилируем регулярные выражения, чтобы ускорить их применение. Здесь '\D' удаляет все символы, не являющиеся цифрами, в столбце 'year_of_release'.

magic_wand = re.compile(r'\D') 
#df['year_of_release'] = df['year_of_release'].map(lambda x: magic_wand.sub('', x))

In [144]:
#df_release=df['year_of_release'].value_counts(dropna=False).to_frame(name='Количество')
#df_release['Процент (%)'] = (df_release['Количество'] / len(df) * 100).round(2)

In [145]:
#print("\nСтатистика по 'year_of_release':")
#print(df_release.head(10))


Статистика по 'year_of_release':
      Количество  Процент (%)
2019       30023        76.09
2018        9078        23.01
2017         324         0.82
2016          30         0.08
2014           2         0.01
2013           1         0.00
2011           1         0.00


На авторынке Казахстана в 2019 году 76% проданных авто - 2019 года выпуска, 23% - 2018 года выпуска. 

### Наименования компаний-дилеров

In [11]:
# посмотрим уникальные значения
df['company'].unique()

array(['Mercur Auto', 'Автоцентр-Бавария', 'БИПЭК АВТО', 'Вираж',
       'Eurasia Motor Premium', 'Daewoo Bus Kazakhstan', 'Caspian Motors',
       'Каспиан Моторс', 'СемАЗ', 'Astana Motors', 'Astana Motors ',
       'Hyundai Com Trans Kazakhstan ', 'Nissan Manufacturing RUS',
       'СВС-ТРАНС', 'Allur Auto', 'Almaty Motors Premium', 'TERRA MOTORS',
       'ТК КАМАЗ', 'Toyota Motor Kazakhstan', 'Лифан Моторс Рус',
       'MAN Truck & Bus Kazakhstan', 'Автомир ГК', 'Autokapital',
       'Автокапитал', 'ММС Рус', 'MMC RUS', 'Mercur Autos',
       'УзАвто-Казахстан', 'Равон Моторс Казахстан',
       'Ravon Motors Kazakstan', 'Renault Россия', 'Scania Central Asia',
       'Subaru Kazakhstan', 'Volkswagen Group Rus', 'Автодом Motors KST',
       'Автомир-Центр', 'ТОО "Eurasia Motor Zhaik"',
       'Scandinavian Motors', 'Хино Моторс Казахстан', 'Hino Motors '],
      dtype=object)

In [12]:
# приводим всё к строке и удаляем лишние пробелы
df['company'] = df['company'].astype(str).str.strip()

По данным сайта 2gis.kz, наименования 'Mercur Autos' и 'Mercur Auto', 'Автомир ГК' и 'Автомир-Центр' относятся к одной и той же компании. Также дублируются наименования на русском и английском языках: 'Хино Моторс Казахстан' и 'Hino Motors', 'Каспиан Моторс' и 'Caspian Motors', 'Автокапитал' и 'Autokapital'.
Изменим значения столбца 'company'

In [13]:
df['company']= df['company'].replace({'Хино Моторс Казахстан':'Hino Motors', 'Каспиан Моторс':'Caspian Motors', 'Автокапитал':'Autokapital', 'Mercur Autos':'Mercur Auto', 'Автомир ГК':'Автомир-Центр'})

In [14]:
df_company=df['company'].value_counts(dropna=False).to_frame(name='Количество')
df_company['Процент (%)'] = (df_company['Количество'] / len(df) * 100).round(2)

In [15]:
print("\nСтатистика по 'company':")
print(df_company)


Статистика по 'company':
                              Количество  Процент (%)
Toyota Motor Kazakhstan            11951        29.90
Astana Motors                      10169        25.44
БИПЭК АВТО                          3624         9.07
Вираж                               2909         7.28
Allur Auto                          2526         6.32
Renault Россия                      1796         4.49
Nissan Manufacturing RUS            1149         2.87
Равон Моторс Казахстан               947         2.37
ТК КАМАЗ                             893         2.23
Mercur Auto                          643         1.61
Ravon Motors Kazakstan               559         1.40
Subaru Kazakhstan                    503         1.26
ММС Рус                              408         1.02
Лифан Моторс Рус                     233         0.58
Автоцентр-Бавария                    224         0.56
СВС-ТРАНС                            202         0.51
Hyundai Com Trans Kazakhstan         155         0.39
Уз

In [16]:
# посмотрим уникальные значения
df['brand'].unique()

array(['Audi', 'BMW', 'Chevrolet', 'Daewoo', 'Ford', 'Foton', 'GAZ',
       'Hyundai', 'Hyundai Truck & Bus', 'Infiniti', 'Isuzu', 'Iveco',
       'Jac', 'Jaguar', 'KAMAZ', 'Kia', 'Lada', 'Land Rover', 'Lexus',
       'Lifan', 'MAN', 'Mazda', 'Mercedes-Benz', 'Mercedes-Benz Trucks',
       'Mini', 'Mitsubishi', 'Nissan', 'Peugeot', 'Porsche', 'Ravon',
       'Renault', 'Scania', 'Skoda', 'Subaru', 'Toyota', 'UAZ',
       'Volkswagen', 'Volvo', 'Урал', 'Cadillac', 'Shacman', 'Hino',
       'Dong Feng', 'ANKAI', 'Nefaz', 'Yutong'], dtype=object)

In [17]:
df_brand=df['brand'].value_counts(dropna=False).to_frame(name='Количество')
df_brand['Процент (%)'] = (df_brand['Количество'] / len(df) * 100).round(2)

In [18]:
print("\nСтатистика по 'brand':")
print(df_brand)


Статистика по 'brand':
                      Количество  Процент (%)
Toyota                     10745        26.89
Hyundai                    10168        25.44
Jac                         1991         4.98
Lada                        1969         4.93
GAZ                         1821         4.56
Renault                     1796         4.49
Ravon                       1794         4.49
Lexus                       1206         3.02
Nissan                      1053         2.63
UAZ                          927         2.32
KAMAZ                        892         2.23
Kia                          832         2.08
Volkswagen                   750         1.88
Chevrolet                    635         1.59
Subaru                       504         1.26
Mitsubishi                   410         1.03
Skoda                        379         0.95
ANKAI                        300         0.75
Lifan                        233         0.58
BMW                          209         0.52
Isuzu     

In [19]:
df['model'].unique()

array(['A3', 'A4', 'A6', 'A7', 'A8', 'Q5', 'Q7', '3 серия', '5 серия',
       '6 серия', '7 серия', 'X1', 'X3', 'X4', 'X5', 'X6', 'X7', 'Z4',
       'Aveo', 'Captiva', 'Niva', 'Tahoe', 'Tracker', 'BS 106 A',
       'Fiesta', 'Mondeo', 'BJ3253DMPKB-AD', '2217', '2310', '2705',
       '2752', '3221', '3302', '3308', '3309', 'Next', 'Accent', 'Creta',
       'Elantra', 'Santa Fe', 'Sonata', 'TQ-1/H-1', 'County', 'H350',
       'HD 160', 'HD 170', 'HD 65', 'HD 78', 'Universe', 'Tucson', 'Q50',
       'QX50', 'QX60', 'QX80', 'D-MAX', 'NMR', 'NPR', 'Daily', 'Daily A',
       'Daily V', 'Stralis', 'Tipper', 'S3', 'S5', 'F-Pace', 'F-Type',
       'XE', 'XF', 'XJ', '4308', '4311', '43118', '43253', '43502',
       '44108', '45142', '45143', '53215', '53504', '53605', '54115',
       '5490', '65111', '65115', '65116', '65117', '6520', '65206',
       '65225', '66052', '6606', 'Cerato', 'Optima', 'Picanto', 'Quoris',
       'Rio', 'Sorento', 'Soul', 'Sportage', '4x4', 'Granta', 'Kalina',
       '

In [20]:
df['country_of_manufacture'].unique()

array(['Германия', 'США', 'Австрия', 'Республика Казахстан',
       'Российская Федерация', 'Корея', 'Япония', 'Таиланд', 'Китай',
       'UK', 'Узбекистан', 'Венгрия', 'Турция', 'Испания', 'Нидерланды',
       'Польша', 'Швеция', 'Белоруссия', 'Бельгия'], dtype=object)

кодируем страны в столбце 'страна производства' в alpha3 (RUS, KAZ, USA и т.д.)


In [21]:
df['country_of_manufacture'] = df['country_of_manufacture'].replace({
    'Германия': 'DEU',
    'США': 'USA',
    'Австрия': 'AUT', 
    'Республика Казахстан': 'KAZ',
    'Российская Федерация': 'RUS',
    'Корея': 'KOR',
    'Япония': 'JPN',
    'Таиланд': 'THA',
    'Китай': 'CHN',
    'UK': 'GBR',
    'Узбекистан': 'UZB',
    'Венгрия': 'HUN',
    'Турция': 'TUR',
    'Испания': 'ESP',
    'Нидерланды': 'NLD',
    'Польша': 'POL',
    'Швеция': 'SWE', 
    'Белоруссия': 'BLR', 
    'Бельгия': 'BEL'
})    

In [22]:
df_country=df['country_of_manufacture'].value_counts(dropna=False).to_frame(name='Количество')
df_country['Процент (%)'] = (df_country['Количество'] / len(df) * 100).round(2)

In [23]:
print("\nСтатистика по 'country_of_manufacture':")
print(df_country)


Статистика по 'country_of_manufacture':
     Количество  Процент (%)
KAZ       19369        48.46
RUS       12846        32.14
JPN        4595        11.50
TUR        1167         2.92
THA         970         2.43
USA         289         0.72
DEU         265         0.66
GBR         197         0.49
CHN          94         0.24
UZB          70         0.18
AUT          33         0.08
SWE          18         0.05
KOR          17         0.04
NLD          12         0.03
ESP           9         0.02
POL           6         0.02
BEL           6         0.02
HUN           2         0.01
BLR           1         0.00


Почти половина всех автомобилей, представленных на рынке Казахстана, - автомобили местного производства - 48.46%. На втором месте - авто из России (32.14%) 

### Обработка значений столбца 'тип топлива'

In [24]:
df['fuel'].unique()

array(['Бензин', 'Дизель', nan, 'дизель ', 'бензин ', 'гибрид', '2',
       '1,6', '0', 'Электро', 'Электричество'], dtype=object)

In [25]:
df[df['fuel']=='2']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
23723,2019,Январь,Renault Россия,Renault,Duster,2019,RUS,2,MT,4WD,4WD,Уральск,1.0,12249.61125,12249.61125,Западно-Казахстанская область,Внедорожники,Субкомпактные SUV
23724,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,AT,4WD,4WD,Нур-Султан,1.0,12249.61125,12249.61125,г.Нур-Султан,Внедорожники,Субкомпактные SUV
23725,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,AT,4WD,4WD,Нур-Султан,1.0,12249.61125,12249.61125,г.Нур-Султан,Внедорожники,Субкомпактные SUV
23726,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,AT,4WD,4WD,Кызылорда,1.0,12249.61125,12249.61125,Кызылординская область,Внедорожники,Субкомпактные SUV
23727,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,MT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23729,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,AT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23730,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,AT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23731,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,MT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23732,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,MT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23746,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,AT,4WD,4WD,Усть-Каменогорск,1.0,12249.61125,12249.61125,Восточно-Казахстанская область,Внедорожники,Субкомпактные SUV


In [26]:
df[df['fuel']=='1,6']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
23728,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",MT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23733,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",MT,4WD,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23734,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",AT,Передний,Передний,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23735,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",AT,Передний,Передний,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23736,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",AT,Передний,Передний,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23751,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",AT,Передний,Передний,Усть-Каменогорск,1.0,12249.61125,12249.61125,Восточно-Казахстанская область,Внедорожники,Субкомпактные SUV
23754,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",MT,4WD,4WD,Алматы,1.0,12249.61125,12249.61125,г.Алматы,Внедорожники,Субкомпактные SUV
23763,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",MT,4WD,4WD,Нур-Султан,1.0,12249.61125,12249.61125,г.Нур-Султан,Внедорожники,Субкомпактные SUV
23764,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6",MT,4WD,4WD,Петропавловск,1.0,12249.61125,12249.61125,Северо-Казахстанская область,Внедорожники,Субкомпактные SUV
24997,2019,Январь,Renault Россия,Renault,Sandero,2018,RUS,"1,6",0,0,0,Атырау,1.0,12476.89637,12476.89637,Атырауская область,Легковые автомобили,B класс


В строках Renault Duster заметно смещение значений в столбцах fuel, engine_volume, transmission_box

In [42]:
# запишем данные в отдельную переменную
df_duster=df[((df['fuel']=='2') | (df['fuel']=='1,6'))]

In [43]:
df_duster=df_duster.reset_index()

In [44]:
# сохраним индексы в список df_duster_index
df_duster_index = list(df_duster['index'])

In [51]:
# для каждого индекса поменяем значение в столбце "коробка передач" на значения в столбце "объем двигателя"
for i in df_duster_index:
    df.loc[i,'transmission_box']=df.loc[i,'engine_volume']

In [53]:
# аналогично меняем значение в столбце "объем двигателя" на значение в столбце "тип топлива"
for i in df_duster_index:
    df.loc[i,'engine_volume']=df.loc[i,'fuel']

In [55]:
df[((df['fuel']=='2') | (df['fuel']=='1,6'))]

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
23723,2019,Январь,Renault Россия,Renault,Duster,2019,RUS,2,2,MT,4WD,Уральск,1.0,12249.61125,12249.61125,Западно-Казахстанская область,Внедорожники,Субкомпактные SUV
23724,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,2,AT,4WD,Нур-Султан,1.0,12249.61125,12249.61125,г.Нур-Султан,Внедорожники,Субкомпактные SUV
23725,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,2,AT,4WD,Нур-Султан,1.0,12249.61125,12249.61125,г.Нур-Султан,Внедорожники,Субкомпактные SUV
23726,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,2,AT,4WD,Кызылорда,1.0,12249.61125,12249.61125,Кызылординская область,Внедорожники,Субкомпактные SUV
23727,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,2,MT,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23728,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,"1,6","1,6",MT,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23729,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,2,AT,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23730,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,2,AT,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23731,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,2,MT,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV
23732,2019,Январь,Renault Россия,Renault,Duster,2018,RUS,2,2,MT,4WD,Караганда,1.0,12249.61125,12249.61125,Карагандинская область,Внедорожники,Субкомпактные SUV


In [57]:
df[df['fuel']=='0']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
37121,2019,Январь,БИПЭК АВТО,UAZ,3741,2018,KAZ,0,"2,693",5 МТ,Полный,Талдыкорган,1.0,14200.0,14200.0,Алматинская область,Коммерческие автомобили,Малотоннажные грузовики
37122,2019,Январь,БИПЭК АВТО,UAZ,3741,2018,KAZ,0,"2,693",5 МТ,Полный,Актобе,3.0,14200.0,42600.0,Актюбинская область,Коммерческие автомобили,Малотоннажные грузовики
37123,2019,Январь,БИПЭК АВТО,UAZ,3741,2018,KAZ,0,"2,693",5 МТ,Полный,Актау,2.0,14200.0,28400.0,Мангистауская область,Коммерческие автомобили,Малотоннажные грузовики


Renault Duster 1.6 и 2.0 2018 и 2019 года выпуска - с бензиновым двигателем, UAZ 3741 - также работает на бензине

In [60]:
df['fuel']=df['fuel'].fillna('unknown')

In [61]:
df.iloc[37121]

year                                         2019
month                                      Январь
company                                БИПЭК АВТО
brand                                         UAZ
model                                        3741
year_of_release                              2018
country_of_manufacture                        KAZ
fuel                                            0
engine_volume                               2,693
transmission_box                             5 МТ
type_of_drive                              Полный
region                                Талдыкорган
quantity                                      1.0
price                                     14200.0
sale                                      14200.0
oblact                        Алматинская область
segment                   Коммерческие автомобили
class                     Малотоннажные грузовики
Name: 37121, dtype: object

Кодируем тип топлива в (F - fuel, D - diesel, E - electro, HYB - hybrid) и меняем значения в столбце. Пропуски заменим на "неизвестно"


In [62]:
df['fuel'] = df['fuel'].astype(str).str.strip()

In [63]:
df['fuel'] = df['fuel'].str.lower()

In [64]:
# 
df['fuel'] = df['fuel'].replace({'бензин': 'F', '0':'F','1,6':'F','2':'F','дизель': 'D', 'электро': 'E', 'электричество':'E', 'гибрид': 'HYB'})

In [65]:
df['fuel'].unique()

array(['F', 'D', 'unknown', 'HYB', 'E'], dtype=object)

### Тип привода

В автомобилях бывают три основных типа приводов:

1. Передний привод (FWD, Front wheel drive). 
2. Задний привод (RWD, Rear Wheel Drive). 
3. Полный привод (4WD, Four wheel drive). Имеет разновидности:
   - Подключаемый полный привод (Part Time).
   - Постоянный полный привод (Full Time).
   - Автоматически подключаемый полный привод (AWD, All wheel drive).

In [66]:
df['type_of_drive'].unique()

array(['передний', 'FWD', 'quattro', 'Задний', 'Полный', 'AWD',
       'Передний', '4WD', '4х2.2', nan, 'RWD', '2 WD', '2WD', '4 WD',
       '4X4', '4х4', '4x4', '4x2', '4х2', '4X2', 'Астана',
       'Передний (FF)', '0', 'FF', '#Н/Д', 'полный', '4Motion'],
      dtype=object)

В автомобилях бывают 4 основных типа приводов:
    Передний привод (FWD, Front wheel drive). 
    Задний привод (RWD, Rear Wheel Drive).
    Полный привод (4WD, Four wheel drive).
    Автоматически подключаемый полный привод (AWD, All wheel drive).
    Типы привода 2WD:
Передний привод (FWD). В таких автомобилях крутящий момент от двигателя передаётся на передние колёса, в то время как задние вращаются свободно. 
dubizzle.com
avtobeginner.ru
drive2.ru
Задний привод (RWD). 

    quattro — название, используемое компанией AUDI AG для обозначения технологий, либо систем постоянного полного привода. 4Motion (4MOTION) — фирменная система полного привода автомобилей Volkswagen.
    #Н/Д - нет данных
    4x2 или 2WD имеют привод на два колеса (2WD) с четырьмя колесами (
    Автомобиль КамАЗ 65116 имеет задний (FR) привод.
    КамАЗ 45141 имеет полный привод (4WD)
    Автомобиль Renault Duster может иметь следующие типы привода: передний (FF) и полный (4WD). 
    Все версии автомобиля Renault Sandero — с передним приводом


In [67]:
df[df['type_of_drive']=='Астана']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
15771,2019,Май,ТК КАМАЗ,KAMAZ,65116,2019,KAZ,D,NaN,MT,Астана,Нур-Султан,1.0,55655.86316,55655.86316,г.Нур-Султан,Коммерческие автомобили,Крупнотоннажные грузовики
39549,2019,Май,ТК КАМАЗ,KAMAZ,45141,2019,KAZ,D,NaN,MT,Астана,Нур-Султан,1.0,53963.81966,53963.81966,г.Нур-Султан,Коммерческие автомобили,Крупнотоннажные грузовики


In [68]:
df.loc[15771,'type_of_drive'] = 'RWD'

In [69]:
df.loc[39549,'type_of_drive'] = '4WD'

In [70]:
df['type_of_drive']=df['type_of_drive'].astype(str)

In [71]:
df['type_of_drive']=df['type_of_drive'].str.upper()

In [72]:
# создадим функцию, распределяющую значения столбца "тип привода" на 6 основных категорий
def drive_type(value):
    if value in ['ПЕРЕДНИЙ','ПЕРЕДНИЙ (FF)','FF', 'FWD']:
        return 'FWD'
    elif value in ['ЗАДНИЙ','RWD']:
        return 'RWD'
    elif '2' in value:
        return '2WD'
    elif 'AWD' in value:
        return value    
    elif value in ['QUATTRO', '4MOTION', '4WD', '4 WD', '4X4', '4Х4', 'ПОЛНЫЙ']: 
        return '4WD'
    return 'unknown'

In [73]:
df['type_of_drive'] = df['type_of_drive'].apply(drive_type)

In [74]:
df['type_of_drive'].value_counts(normalize=True)

4WD        0.351824
2WD        0.245809
FWD        0.226067
unknown    0.110319
RWD        0.041911
AWD        0.024070
Name: type_of_drive, dtype: float64

In [75]:
df[df['type_of_drive']=='unknown']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
861,2019,Март,СемАЗ,Foton,BJ3253DMPKB-AD,2019,KAZ,unknown,NaN,NaN,unknown,Алматы,1.0,48275.86207,48275.86207,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
862,2019,Июнь,СемАЗ,Foton,BJ3253DMPKB-AD,2018,KAZ,unknown,NaN,NaN,unknown,Усть-Каменогорск,1.0,49735.91972,49735.91972,Восточно-Казахстанская область,Коммерческие автомобили,Крупнотоннажные грузовики
863,2019,Июнь,СемАЗ,Foton,BJ3253DMPKB-AD,2018,KAZ,unknown,NaN,NaN,unknown,Кокшетау,1.0,48784.39304,48784.39304,Акмолинская область,Коммерческие автомобили,Крупнотоннажные грузовики
864,2019,Июль,СемАЗ,Foton,BJ3253DMPKB-AD,2018,KAZ,unknown,NaN,NaN,unknown,Алматы,5.0,48725.42295,243627.11480,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
865,2019,Август,СемАЗ,Foton,BJ3253DMPKB-AD,2018,KAZ,unknown,NaN,NaN,unknown,Алматы,2.0,31031.65229,62063.30457,г.Алматы,Коммерческие автомобили,Крупнотоннажные грузовики
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39961,2019,Сентябрь,ТК КАМАЗ,KAMAZ,58815Z,2019,KAZ,D,NaN,MT,unknown,Актобе,1.0,48756.92071,48756.92071,Актюбинская область,Коммерческие автомобили,Крупнотоннажные грузовики
39962,2019,Сентябрь,ТК КАМАЗ,KAMAZ,58815Z,2019,KAZ,D,NaN,MT,unknown,Актобе,1.0,48756.92071,48756.92071,Актюбинская область,Коммерческие автомобили,Крупнотоннажные грузовики
39963,2019,Сентябрь,ТК КАМАЗ,KAMAZ,58815Z,2019,KAZ,D,NaN,MT,unknown,Актобе,1.0,48756.92071,48756.92071,Актюбинская область,Коммерческие автомобили,Крупнотоннажные грузовики
39964,2019,Сентябрь,ТК КАМАЗ,KAMAZ,58815Z,2019,KAZ,D,NaN,MT,unknown,Актобе,1.0,48756.92071,48756.92071,Актюбинская область,Коммерческие автомобили,Крупнотоннажные грузовики


### Обработка значений столбца "объем двигателя"

приведем в порядок столбец объем двигателя (заменим запятую на точку, уберем букву L, разберемся с мотором 400 литров у ягуара (там перепутано объем и мощность двигателя - тем кто это найдет и определит нормальный литраж двигателя можно значок давать за въедливость, а можно просто удалить)) + есть неправильные объемы двигателя для Шевроле Нива - 21.6, 20.6 и так далее


In [77]:
df['engine_volume'].unique()

array(['1,4', '2', '3', '4,4', '6,6', '1,5', '1,596', '2,4', '1,7',
       '26,7', '25,7', '24,7', '23,7', '22,7', '21,7', '20,7', '19,7',
       '18,7', '17,7', '16,7', '15,7', '14,7', '13,7', '12,7', '11,7',
       '10,7', '9,7', '8,7', '7,7', '6,7', '5,7', '4,7', '3,7', '2,7',
       '6,2', '1,4 Turbo', '2,5', '1,6', nan, '2,8', '4,3', '4,3,',
       '1,6 MPI', '2,0 MPI', '2,4 GDI', '2,4 MPI', '2,5 CRDI VGT',
       '2,5 CRDI WGT', '3,9', '7,5', '12,3', '1,6 T-GDI', '2,0 CRDI',
       '2.0', '3,5', '5,6', '5,2', '3,0 L', '1.6', '1.5', '5', '1,598',
       '1,248', '1,998', '2,359', '1,999', '3,342', '1,591', '3,47',
       '1,69', '1,774', '2.5', '2.7 ', '3.5', '2.7', '4.6', '4,6', '5.7',
       '1,8', '10,5', '4', '5,5', '12', '12,8', '11', '2,2', '1,2',
       '1,485', '13', '1,33', '#Н/Д', '4.0', '2.4G', '2,4G', '2.8',
       '2,693', '2,0', '3,8', '3.8', '1,2T', '3,6', '7,6', '4,9', '2.0h',
       '2,9', '400 Л.С.', '4,98 L,', '4,98', '88 KWH', '8,4 L,', '6,7L',
       '6,5'], d

In [93]:
df[df['engine_volume'] =='26,7']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
399,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,"26,7",MT,4WD,Костанай,1.0,10536.24452,10536.24452,Костанайская область,Внедорожники,Субкомпактные SUV


In [92]:
df[((df['model']=='Niva') & (df['engine_volume'] !='1,7'))]

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
399,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,"26,7",MT,4WD,Костанай,1.0,10536.24452,10536.24452,Костанайская область,Внедорожники,Субкомпактные SUV
400,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,"25,7",MT,4WD,Атырау,1.0,10536.24452,10536.24452,Атырауская область,Внедорожники,Субкомпактные SUV
401,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,"24,7",MT,4WD,Костанай,1.0,10536.24452,10536.24452,Костанайская область,Внедорожники,Субкомпактные SUV
402,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,"23,7",MT,4WD,Рудный,1.0,10536.24452,10536.24452,Костанайская область,Внедорожники,Субкомпактные SUV
403,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,"22,7",MT,4WD,Рудный,1.0,10536.24452,10536.24452,Костанайская область,Внедорожники,Субкомпактные SUV
404,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,"21,7",MT,4WD,Нур-Султан,1.0,10536.24452,10536.24452,г.Нур-Султан,Внедорожники,Субкомпактные SUV
405,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,"20,7",MT,4WD,Караганда,1.0,10536.24452,10536.24452,Карагандинская область,Внедорожники,Субкомпактные SUV
406,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,"19,7",MT,4WD,Костанай,1.0,10536.24452,10536.24452,Костанайская область,Внедорожники,Субкомпактные SUV
407,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,"18,7",MT,4WD,Усть-Каменогорск,1.0,10536.24452,10536.24452,Восточно-Казахстанская область,Внедорожники,Субкомпактные SUV
408,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,"17,7",MT,4WD,Караганда,1.0,10536.24452,10536.24452,Карагандинская область,Внедорожники,Субкомпактные SUV


Объем двигателя Шевроле Нива составляет от 1.7 до 1.8 л.

In [94]:
# для каждого индекса c 399 по 423 поменяем значение в столбце "объем двигателя"
for i in range(399,424):
    df.loc[i,'engine_volume']=1.7

In [95]:
df[((df['model']=='Niva') & (df['engine_volume'] !='1,7'))]

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
399,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,1.7,MT,4WD,Костанай,1.0,10536.24452,10536.24452,Костанайская область,Внедорожники,Субкомпактные SUV
400,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,1.7,MT,4WD,Атырау,1.0,10536.24452,10536.24452,Атырауская область,Внедорожники,Субкомпактные SUV
401,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,1.7,MT,4WD,Костанай,1.0,10536.24452,10536.24452,Костанайская область,Внедорожники,Субкомпактные SUV
402,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,1.7,MT,4WD,Рудный,1.0,10536.24452,10536.24452,Костанайская область,Внедорожники,Субкомпактные SUV
403,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,1.7,MT,4WD,Рудный,1.0,10536.24452,10536.24452,Костанайская область,Внедорожники,Субкомпактные SUV
404,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,1.7,MT,4WD,Нур-Султан,1.0,10536.24452,10536.24452,г.Нур-Султан,Внедорожники,Субкомпактные SUV
405,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,1.7,MT,4WD,Караганда,1.0,10536.24452,10536.24452,Карагандинская область,Внедорожники,Субкомпактные SUV
406,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,1.7,MT,4WD,Костанай,1.0,10536.24452,10536.24452,Костанайская область,Внедорожники,Субкомпактные SUV
407,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,1.7,MT,4WD,Усть-Каменогорск,1.0,10536.24452,10536.24452,Восточно-Казахстанская область,Внедорожники,Субкомпактные SUV
408,2019,Апрель,Вираж,Chevrolet,Niva,2019,KAZ,F,1.7,MT,4WD,Караганда,1.0,10536.24452,10536.24452,Карагандинская область,Внедорожники,Субкомпактные SUV


In [78]:
df[df['engine_volume']=='88 KWH']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
39818,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,2019,KAZ,E,88 KWH,NaN,2WD,Костанай,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39819,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,2019,KAZ,E,88 KWH,NaN,2WD,Костанай,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39820,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,2019,KAZ,E,88 KWH,NaN,2WD,Костанай,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39821,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,2019,KAZ,E,88 KWH,NaN,2WD,Костанай,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39822,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,2019,KAZ,E,88 KWH,NaN,2WD,Костанай,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39823,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,2019,KAZ,E,88 KWH,NaN,2WD,Костанай,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39824,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,2019,KAZ,E,88 KWH,NaN,2WD,Костанай,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39825,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,2019,KAZ,E,88 KWH,NaN,2WD,Костанай,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39826,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,2019,KAZ,E,88 KWH,NaN,2WD,Костанай,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы
39827,2019,Июль,Allur Auto,ANKAI,HFF6124G03EV3,2019,KAZ,E,88 KWH,NaN,2WD,Костанай,1.0,307986.3633,307986.3633,Костанайская область,Коммерческие автомобили,Большие автобусы


In [79]:
df[df['engine_volume']=='400 Л.С.']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
39544,2019,Февраль,Almaty Motors Premium,Jaguar,I-Pace,2019,GBR,E,400 Л.С.,РЕДУКТОР,4WD,Алматы,1.0,117236.9729,117236.9729,г.Алматы,Внедорожники,Компактные SUV
39545,2019,Апрель,Almaty Motors Premium,Jaguar,I-Pace,2019,GBR,E,400 Л.С.,РЕДУКТОР,4WD,Алматы,1.0,105388.7924,105388.7924,г.Алматы,Внедорожники,Компактные SUV
39546,2019,Июнь,Almaty Motors Premium,Jaguar,I-Pace,2019,GBR,E,400 Л.С.,РЕДУКТОР,4WD,Алматы,1.0,120413.2793,120413.2793,г.Алматы,Внедорожники,Компактные SUV



У Jaguar I-Pace нет двигателя с указанием объёма, так как автомобиль является электромобилем. Максимальная мощность 400 л.с. (294 кВт). 
Мощность электродвигателя автомобиля ANKAI HFF6124G03EV3 - 88 кВтч.

In [97]:
df['engine_volume'].unique()

array(['1,4', '2', '3', '4,4', '6,6', '1,5', '1,596', '2,4', '1,7', 1.7,
       '6,2', '1,4 Turbo', '2,5', '1,6', nan, '2,7', '2,8', '4,3', '4,3,',
       '1,6 MPI', '2,0 MPI', '2,4 GDI', '2,4 MPI', '2,5 CRDI VGT',
       '2,5 CRDI WGT', '3,9', '7,5', '12,3', '1,6 T-GDI', '2,0 CRDI',
       '2.0', '3,5', '5,6', '8,7', '5,2', '3,0 L', '1.6', '1.5', '5',
       '1,598', '1,248', '1,998', '2,359', '1,999', '3,342', '1,591',
       '3,47', '1,69', '1,774', '2.5', '2.7 ', '3.5', '2.7', '4.6', '4,6',
       '5.7', '5,7', '1,8', '10,5', '4', '5,5', '12', '12,8', '11', '2,2',
       '1,2', '1,485', '13', '1,33', '#Н/Д', '4.0', '2.4G', '2,4G', '2.8',
       '2,693', '2,0', '3,8', '3.8', '1,2T', '3,6', '7,6', '4,9', '2.0h',
       '2,9', '400 Л.С.', '4,98 L,', '4,98', '88 KWH', '8,4 L,', '6,7L',
       '6,5'], dtype=object)

In [102]:
df['engine_volume'] = df['engine_volume'].replace({
    '400 Л.С.': '294 кВт',
    '88 KWH': '88 кВт',
    ',': '.',
    ' Turbo': '',
    ' MPI': '',
    ' GDI': '',
    ' CRDI VGT': '',
    ' CRDI WGT': '',
    ' T-GDI': '',
    ' CRDI': '',
    ' L': '',
    'L': '',
    '#Н/Д': 'unknown',
    'G': '',
    'T': '',
    'h': ''})

In [99]:
df['engine_volume'].unique()

array(['1,4', '2', '3', '4,4', '6,6', '1,5', '1,596', '2,4', '1,7', 1.7,
       '6,2', '1,4 Turbo', '2,5', '1,6', nan, '2,7', '2,8', '4,3', '4,3,',
       '1,6 MPI', '2,0 MPI', '2,4 GDI', '2,4 MPI', '2,5 CRDI VGT',
       '2,5 CRDI WGT', '3,9', '7,5', '12,3', '1,6 T-GDI', '2,0 CRDI',
       '2.0', '3,5', '5,6', '8,7', '5,2', '3,0 L', '1.6', '1.5', '5',
       '1,598', '1,248', '1,998', '2,359', '1,999', '3,342', '1,591',
       '3,47', '1,69', '1,774', '2.5', '2.7 ', '3.5', '2.7', '4.6', '4,6',
       '5.7', '5,7', '1,8', '10,5', '4', '5,5', '12', '12,8', '11', '2,2',
       '1,2', '1,485', '13', '1,33', 'unknown', '4.0', '2.4G', '2,4G',
       '2.8', '2,693', '2,0', '3,8', '3.8', '1,2T', '3,6', '7,6', '4,9',
       '2.0h', '2,9', '294 кВт', '4,98 L,', '4,98', '88 кВт', '8,4 L,',
       '6,7L', '6,5'], dtype=object)

Обработка данных в столбце "тип трансмиссии"

уберем неявные дубликаты (для простоты оставим два типа трансмиссии: механика и автомат)


In [81]:
df['transmission_box'].unique()

array(['S-tronic', 'S-Tronic', 'Tiptronic', 'АКПП', 'АКП', 'Steptronic',
       'AT', '6 АТ', '5 МТ', '6АТ', 'MT', '8AT', '5МТ', 'Powershift S6',
       '6AT', nan, 'Мех.', '6 AT', '6 MT', '6MT', '5 AT', '5AT', 'МКПП',
       '7 DCT', '7DCT', '8 AT', 'МТ', 'CVT', 'DCT', '4 АТ', '6 МТ', '6МТ',
       '4АТ', '4AT', '8АТ', '8 АТ', '5MT', 'AMT', 'CVT (вариатор)',
       'CVT (вариATор)', 'CVT(вAриATор)', '7G-TRONIC', '9G-TRONIC',
       '12AT', '12АТ', '5АТ', '7АТ', '7AT', '6M/T', 'PDK', '8-ступ АКПП',
       '8', '8АКПП', '8 АКПП Tiptronic S', '7 АКПП (PDK)', '7 АКПП PDK',
       '8 АКПП (PDK)', 'АТ', 'АT', 'МT', '0', '7 DSG', '7DSG', '6 DSG',
       '6DSG', '#Н/Д', 'DSG', '6A', '6А', '5M', 'A/T', 'M/T', 'МКП', '5М',
       'М/T', '7 АКП', '7АКП', 'А/T', '8A', 'TDI', '9AT', 'РЕДУКТОР'],
      dtype=object)

    Существует автоматическая коробка передач и механическая. В группе автоматики мы можем найти три типа: DCT, CVT и AMT.
    S-Tronic — это механическая коробка передач с бесступенчатым изменением передаточного числа
    Tiptronic и Steptronic— это автоматическая коробка передач 
    PDK (нем. Porsche DoppelKupplungsgetriebe, буквально — «с двойным сцеплением, трансмиссия от Порше») — автоматическая коробка передач с двойным сцеплением разработки компании Porsche AG. 
    TDI не относится к типу коробки передач, это обозначение двигателя. 
    Редуктор не относится к механическим или автоматическим коробкам передач, это отдельный элемент трансмиссии автомобиля. 
    Вариатор (CVT) — это вариант автоматической трансмиссии, полностью лишённый сцеплений. Переключения передач в этой конструкции не предусмотрено. 
    7G-TRONIC — это автоматическая коробка передач (АКПП). 
    Powershift 6DCT450 (MPS6) — это роботизированная коробка передач. Однако по конструкции она механическая коробка, которая содержит пару входных и пару выходных валов
    6DSG не является «механикой» в привычном понимании, а представляет собой роботизированную коробку передач. По своему строению она сочетает в себе надёжность механической коробки передач и простоту автоматической. Переключение передач в ней осуществляется вручную, а непосредственный процесс работы коробки происходит полностью автоматически.

In [246]:
#for x in df['transmission_box']:
   # x=re.sub("[^A-Za-z]", "", str(x))

In [82]:
df['transmission_box']=df['transmission_box'].astype(str).str.upper()

In [83]:
df[df['transmission_box']=='8']

,year,month,company,brand,model,year_of_release,country_of_manufacture,fuel,engine_volume,transmission_box,type_of_drive,region,quantity,price,sale,oblact,segment,class
21879,2019,Февраль,Mercur Auto,Porsche,Cayenne,2018,DEU,F,3,8,4WD,Алматы,1.0,90000.00000,90000.00000,г.Алматы,Внедорожники,Полноразмерные SUV
21880,2019,Февраль,Mercur Auto,Porsche,Cayenne,2018,DEU,F,3,8,4WD,Алматы,1.0,121474.89640,121474.89640,г.Алматы,Внедорожники,Полноразмерные SUV
21881,2019,Февраль,Mercur Auto,Porsche,Cayenne,2018,DEU,F,3,8,4WD,Алматы,1.0,132373.53870,132373.53870,г.Алматы,Внедорожники,Полноразмерные SUV
21882,2019,Февраль,Mercur Auto,Porsche,Cayenne,2018,DEU,F,3,8,4WD,Алматы,1.0,114165.97620,114165.97620,г.Алматы,Внедорожники,Полноразмерные SUV
21883,2019,Февраль,Mercur Auto,Porsche,Cayenne,2018,DEU,F,3,8,4WD,Алматы,1.0,96249.73469,96249.73469,г.Алматы,Внедорожники,Полноразмерные SUV


In [250]:
# скомпилируем регулярные выражения, чтобы ускорить их применение. Здесь '[^a-z|а-яё]' удаляет все символы,
# не являющиеся буквами, в столбце 'transmission_box'.

#magic_wand = re.compile('[^a-z|а-яё]') 
#df['transmission_box'] = df['transmission_box'].map(lambda x: magic_wand.sub('', x))

In [ ]:
transmission_m = ['S-TRONIC','5 МТ', 'MT', '5МТ', 'POWERSHIFT S6',
       'МЕХ.', '6 MT', '6MT', 'МКПП', '7 DCT',
       '7DCT', '8 AT', 'МТ', '6 МТ', '6МТ', 
        '5MT', '6M/T', 'МT', '4WD', 'ПЕРЕДНИЙ', '0', '7 DSG',
       '7DSG', '6 DSG', '6DSG', '#Н/Д', 'DSG', '6A', '6А', '5M', 'A/T',
       'M/T', 'МКП', '5М', 'М/T', '7 АКП', '7АКП', 'А/T', '8A', 'TDI',
       '9AT', 'РЕДУКТОР']

In [309]:
df['transmission_box'].unique()

array(['S-TRONIC', 'TIPTRONIC', 'АКПП', 'АКП', 'STEPTRONIC', 'AT', '6 АТ',
       '5 МТ', '6АТ', 'MT', '8AT', '5МТ', 'POWERSHIFT S6', '6AT', 'NAN',
       'МЕХ.', '6 AT', '6 MT', '6MT', '5 AT', '5AT', 'МКПП', '7 DCT',
       '7DCT', '8 AT', 'МТ', 'CVT', 'DCT', '4 АТ', '6 МТ', '6МТ', '4АТ',
       '4AT', '8АТ', '8 АТ', '5MT', 'AMT', 'CVT (ВАРИАТОР)',
       'CVT (ВАРИATОР)', 'CVT(ВAРИATОР)', '7G-TRONIC', '9G-TRONIC',
       '12AT', '12АТ', '5АТ', '7АТ', '7AT', '6M/T', 'PDK', '8-СТУП АКПП',
       '8', '8АКПП', '8 АКПП TIPTRONIC S', '7 АКПП (PDK)', '7 АКПП PDK',
       '8 АКПП (PDK)', 'АТ', 'АT', 'МT', '4WD', 'ПЕРЕДНИЙ', '0', '7 DSG',
       '7DSG', '6 DSG', '6DSG', '#Н/Д', 'DSG', '6A', '6А', '5M', 'A/T',
       'M/T', 'МКП', '5М', 'М/T', '7 АКП', '7АКП', 'А/T', '8A', 'TDI',
       '9AT', 'РЕДУКТОР'], dtype=object)